# User-defined types

Sometimes you want a new kind of thing. In Julia types are just some data which we can add new behaviors to. New behavior is added by defining methods for existing and new generic functions. In this example, we're going to define a modular integer type called `ModInt`, which does "clock arithmetic" – with some number of "hours".

## Example: `ModInt`

In [ ]:
struct ModInt{n} <: Integer
    k::Int

    ModInt{n}(k) where {n} = new(mod(k,n))
end

In [ ]:
a = ModInt{13}(213)

In [ ]:
b = ModInt{13}(987)

In [ ]:
dump(a)

That's it – that's all there is to creating a new type. Of course, it can't do much yet...

In [ ]:
a + b

## Customizing behavior

First, some nicer display:

In [ ]:
Base.show{n}(io::IO, k::ModInt{n}) = print(io, "$(k.k) mod $n")

In [ ]:
a

In [ ]:
b

In [ ]:
import Base: +, -, *, /, inv

In [ ]:
# basic arithmetic

+(a::ModInt{n}, b::ModInt{n}) where {n} = ModInt{n}(a.k + b.k)
-(a::ModInt{n}, b::ModInt{n}) where {n} = ModInt{n}(a.k - b.k)
*(a::ModInt{n}, b::ModInt{n}) where {n} = ModInt{n}(a.k * b.k)
-(a::ModInt{n}) where {n} = ModInt{n}(-a.k)

/(a::ModInt{n}, b::ModInt{n}) where {n} = a*inv(b)
inv(a::ModInt{n}) where {n} = ModInt{n}(invmod(a.k, n))

# interacting with regular integers

Base.promote_rule(::Type{ModInt{n}}, ::Type{Int}) where {n} = ModInt{n}
Base.convert(::Type{ModInt{n}}, i::Int) where {n} = ModInt{n}(i)

In [ ]:
a + b

In [ ]:
a*b

In [ ]:
(a/b)*b

In [ ]:
inv(b)

In [ ]:
inv(ModInt{15}(5))

With just a bit of arithmetic, we can do quite a bit:

In [ ]:
A = map(ModInt{13}, rand(Int, 10, 10))

That's a bit verbose. We can tweak `Base.show` a bit to make it less repetetive:

In [ ]:
Base.show{n}(io::IO, k::ModInt{n}) =
    print(io, get(io, :compact, false) ? k.k : "$(k.k) mod $n")

In [ ]:
A

Now let's play around...

In [ ]:
A + A

In [ ]:
2A

In [ ]:
A*A

In [ ]:
A^2

In [ ]:
A^100

In [ ]:
A^1000000

In [ ]:
@which A^1000000

In [ ]:
sizeof(A)

In [ ]:
sizeof(A) ÷ prod(size(A))

In [ ]:
dump(a)

In [ ]:
a.k

In [ ]:
a.k = 6